# Post Result Analysis

In [1]:
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt

%matplotlib inline

**Original Paper's last paragraph of page 8 and the first paragraph of page 9** [here](https://arxiv.org/pdf/1701.05927.pdf#8).<br><br>
One of the unique properties of high energy particle physics is that we have a library of useful jet observables that are physically motivated functions $f : \mathbb{R}^{25×25} → \mathbb{R}$ whose features are qualitatively (and in some cases, quantitatively) well understood. We can use the distributions of these observables to assess the abilty of the GAN to mimic Pythia. Three such features of a jet image ***$I$*** are the mass ***$m$***, transverse momentum ***$p_{T}$***, and n-subjettiness ***$\tau_{21}$***:

$$p^2_T(I) = (\sum_{i}I_icos(\phi_i))^2 + (\sum_{i}I_isin(\phi_i))^2$$
$$m^2(I) = (\sum_{i}I_icosh(\eta_i))^2 - p^2_T(I) - (\sum_{i}I_isinh(\eta_i))^2$$
$$\tau_n(I) \propto \sum_{i}argmin_a(I_i)*(\sqrt{(\eta_i-\eta_a)^2-(\phi_i-\phi_a)^2})$$
$$\tau_{21}(I) = \frac{\tau_2(I)}{\tau_1(I)}$$

where ***$I_i$***, ***$\eta_i$***, and ***$\phi_i$*** are the pixel intensity, pseudorapidity, and azimuthal angle, respectively. The sums run over the entire image. The quantities $\eta_a$ and $\phi_a$ are axis values determined with the one-pass $k_t$ axis selection using the winner-take-all combination scheme.

In [2]:
grid = 0.5*np.linspace(-1.25, 1.25, 26)[:-1] + np.linspace(-1.25, 1.25, 26)[1:]
eta = np.tile(grid, (25, 1))
phi = np.tile(grid[::-1].reshape(-1, 1), (1, 25))

In [3]:
image = np.abs(np.random.randn(25, 25))

### Discrete Momentum

Calculation of the Jet Momentum from a Pixelated Jet Image

In [4]:
def discrete_momentum(img):
    '''
    Calculates the jet transverse momentum from a pixelated jet image
    
    Args:
    - img: Jet Images in shape (None, 25, 25)
    Returns:
    - pt_2: Jet Transverse Momentum
    '''
    P_x = np.sum(img * np.cos(phi), axis=(1, 2))
    P_y = np.sum(img * np.sin(phi), axis=(1, 2))
    return np.sqrt(np.square(P_x) + np.square(P_y))

### Discrete Mass
Calculation of the Jet Mass from a Pixelated Jet Image

In [5]:
def discrete_mass(img):
    '''
    Calculates the jet mass from a pixelated jet image
    
    Args:
    - img: Jet Images in shape (None, 25, 25)
    Returns:
    - M: Jet Mass
    '''
    P_x = np.sum(img * np.cos(phi), axis=(1, 2))
    P_y = np.sum(img * np.sin(phi), axis=(1, 2))
    P_z = np.sum(img * np.sinh(eta), axis=(1, 2))
    E   = np.sum(img * np.cosh(eta), axis=(1, 2))
    
    P_t2 = np.square(P_x) + np.square(P_y)
    M_2  = np.square(E) - (P_t2 + np.square(P_z))
    
    return np.sqrt(M_2)

### N-subjettiness

Calculation of N-subjettiness which is a jet shape designed to identify boosted hadronic objects such as top quarks.[here](https://arxiv.org/abs/1108.2701)

In [6]:
def tau_1(img):
    '''
    Calculates the normalized Tau_1 from a Pixelated Jet Image
    
    Args:
    - img: Jet Images in shape (None, 25, 25)
    Returns:
    - Tau: Normalized Jet Tau1
    '''
    # find coordinate of most energetic pixel
    tau1_axis_eta = eta.ravel()[np.argmax(img)]
    tau1_axis_phi = phi.ravel()[np.argmax(img)]
    
    tau1 = np.sum(img * np.sqrt(
                                np.square(tau1_axis_eta - eta) +
                                np.square(tau1_axis_phi - phi)
                               )
                  )
    return tau1 / np.sum(img, axis=(1,2))

In [7]:
def tau_2(img):   
    '''
    Calculates the normalized Tau_2 from a Pixelated Jet Image
    
    Args:
    - img: Jet Images in shape (None, 25, 25)
    Returns:
    - Tau: Normalized Jet Tau2
    '''
    
    def tau_2_unit(img):
        idx = (img != 0)
        proto = np.stack([img[idx], eta[idx], phi[idx]], axis=-1)

        while proto.shape[0] > 2:
            pair_idx = np.array(list(combinations(range(proto.shape[0]), 2)))

            pt__1 = proto[pair_idx[:,0], 0]
            eta_1 = proto[pair_idx[:,0], 1]
            phi_1 = proto[pair_idx[:,0], 2]

            pt__2 = proto[pair_idx[:,1], 0]
            eta_2 = proto[pair_idx[:,1], 1]
            phi_2 = proto[pair_idx[:,1], 2]

            value = np.min([pt__1, pt__2], axis=0) * ((eta_1 - eta_2)**2 + (phi_1 - phi_2)**2)
            idx1, idx2 = np.sort(pair_idx[np.argmin(value)])

            pt1, eta1, phi1 = proto[idx1]
            pt2, eta2, phi2 = proto[idx2]

            e1, e2 = (pt1 / np.cosh(eta1)), (pt2 / np.cosh(eta2))
            eta_add, pt_add = ((eta1, phi1) if e1 > e2 else (eta2, phi2))
            phi_add = (e1 + e2) * np.cosh(eta_add)
            proto[idx1] = (pt_add, eta_add, phi_add)

            proto = np.delete(proto, idx2, axis=0)

        eta1, eta2 = proto[:, 1].tolist()
        phi1, phi2 = proto[:, 2].tolist()

        grid = np.min([np.sqrt(np.square(eta - eta1) + np.square(phi - phi1)), 
                       np.sqrt(np.square(eta - eta1) + np.square(phi - phi1))], 
                      axis=0)

        return np.sum(img * grid) / np.sum(img)
    
    return np.array(list(map(lambda x: tau_2_unit(x), img)))

In [8]:
def n_subjettiness(img):   
    tau1 = tau_1(img)
    tau2 = tau_2(img) + 1e-60
    
    return tau1/tau2